In [7]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chat = ChatOpenAI(temperature=0.1)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

In [8]:
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

Chat model 은 대화에 최적화 되어 있는데 단지 질문을 받을 뿐 아니라, 대화를 할 수 있다는 뜻이다.

대화는, 그룹 오브 메세지스 즉, 여러 메세지의 묶음이라는 의미이다.

상대로서 대화의 맥락에 맞게 대답할 수 있다는 뜻이다. - Chat Model



만약 model 의 설정을 바꾸고 싶다면 

In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?"
    ),
]

SystemMessage 로 AI 에 일종의 기본 설정, 기본 값, 기본 context 설정을 함.

AIMessage 로 string 을 미리 만들어두고 일종의 가상 대화(마치 AI 가 답변한 것처럼)를 만들었음.

HumanMessage 로 사용자로서 질문을 함.

In [4]:
chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

predict 와 predict_messages 는 string 과 messages 를 predict 하는 차이점이 있다.